# <span style="color:crimson;"><p style="text-align:center;">CONVOLUTIONAL NEURAL NETWORK FROM SCRATCH - FOR BEGINERS

<u><span style="font-size:28px;">WHAT IS CNN:</span></u>

* <p style="font-size:20px;">CNNs, like neural networks, are made up of neurons with learnable weights and biases. Each neuron receives several inputs, takes a weighted sum over them, pass it through an activation function and responds with an output. The whole network has a loss function and all the tips and tricks that we developed for neural networks still apply on CNNs.</p>

* <p style="font-size:20px;">It processes data that has a grid-like arrangement then extracts important features. One huge advantage of using CNNs is that you don't need to do a lot of pre-processing on images.</p>

<u><span style="font-size:28px;">HOW CNN WORKS:</span></u>

![](https://miro.medium.com/max/1750/0*0lrC8RWrShYm8U9L.png)

* <p style="font-size:20px;">When you're working with data in a CNN, each layer returns activation maps. These maps point out important features in the data set. If you gave the CNN an image, it'll point out features based on pixel values, like colors, and give you an activation function.</p>

* <p style="font-size:20px;">Usually with images, a CNN will initially find the edges of the picture. Then this slight definition of the image will get passed to the next layer. Then that layer will start detecting things like corners and color groups. Then that image definition will get passed to the next layer and the cycle continues until a prediction is made.</p>

* <p style="font-size:20px;">The last layer of a CNN is the classification layer which determines the predicted value based on the activation map. If you pass a handwriting sample to a CNN, the classification layer will tell you what letter is in the image.</p>

<u><span style="font-size:28px;">Types of Layers in CNN</span></u>

* <p style="font-size:20px;">INPUT LAYER</p>
* <p style="font-size:20px;">CONVOLUTIONAL LAYER</p>
* <p style="font-size:20px;">ACTIVATION FUNCTION LAYER</p>
* <p style="font-size:20px;">POOLING LAYER</p>
* <p style="font-size:20px;">FULLY-CONNECTED LAYER</p>

<u><span style="font-size:28px;">POOLING LAYERS IN CNN:</span></u>

* <p style="font-size:20px;">The pooling operation involves sliding a two-dimensional filter over each channel of feature map and summarising the features lying within the region covered by the filter.</p>

* <p style="font-size:20px;">For a feature map having dimensions nh x nw x nc, the dimensions of output obtained after a pooling layer is</p>

* <p style="font-size:20px;">(nh - f + 1) / s x (nw - f + 1)/s x nc</p>

                    nh - height of feature map
                    nw - width of feature map
                    nc - number of channels in the feature map
                    f  - size of filter
                    s  - stride length
                    
<u><span style="font-size:28px;">Why to use Pooling Layers?</span></u>

* <p style="font-size:20px;">Pooling layers are used to reduce the dimensions of the feature maps. Thus, it reduces the number of parameters to learn and the amount of computation performed in the network.</p>

* <p style="font-size:20px;">The pooling layer summarises the features present in a region of the feature map generated by a convolution layer. So, further operations are performed on summarised features instead of precisely positioned features generated by the convolution layer. This makes the model more robust to variations in the position of the features in the input image.</p>

<u><span style="font-size:28px;">Types of Pooling Layers</span></u>

<p style="font-size:20px;"><b>MAX POOLING:</b> Max pooling is a pooling operation that selects the maximum element from the region of the feature map covered by the filter. Thus, the output after max-pooling layer would be a feature map containing the most prominent features of the previous feature map.</p>

![](https://media.geeksforgeeks.org/wp-content/uploads/20190721025744/Screenshot-2019-07-21-at-2.57.13-AM.png)

<p style="font-size:20px;"><b>AVERAGE POOLING:</b> Average pooling computes the average of the elements present in the region of feature map covered by the filter. Thus, while max pooling gives the most prominent feature in a particular patch of the feature map, average pooling gives the average of features present in a patch.</p>

![](https://media.geeksforgeeks.org/wp-content/uploads/20190721030705/Screenshot-2019-07-21-at-3.05.56-AM.png)

##### <center> (IMAGE & CONTENT SOURCE - GEEKSFORGEEKS)

-----------

# <span style="color:crimson;"><p style="text-align:center;">CONVOLUTIONAL NEURAL NETWORK IMPLEMENTATION - PYTHON

In [1]:
# IMPORT REQUIRED LIBRARIES:

import tensorflow as tf
from tensorflow import keras
import numpy as np

In [2]:
print(tf.__version__)

2.4.1


In [3]:
#IMPORT FASHION MNIST DATASET:

fashion_mnist = keras.datasets.fashion_mnist

In [4]:
#SPLIT THE DATA INTO TEST AND TRAIN:

(train_images, train_labels), (test_images, test_labels) = fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step


In [5]:
#NORMALIZE THE IMAGES:

train_images = train_images/255.0
test_images = test_images/255.0

In [7]:
#GET THE SHAPE OF THE IMAGES:
train_images[0].shape

(28, 28)

In [8]:
#RESHAPE THE IMAGES FROM 28, 28 TO (28, 28, 1)

train_images = train_images.reshape(len(train_images), 28, 28, 1)
test_images = test_images.reshape(len(test_images), 28, 28, 1)

In [16]:
def build_model(hp):  
    model = keras.Sequential([
    
    # First CNN Layer:
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
        
    # Second CNN Layer:
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    
    # Flatten Layer:    
    keras.layers.Flatten(),
    
    # Dense Layer:    
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    
    #Output Layer    
    keras.layers.Dense(10, activation='softmax')
    ])
    
    #Compile the Model
    model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
    return model

In [17]:
# We are importing randomsearch - this will tell us which parameters will be best:
from kerastuner import RandomSearch
from kerastuner.engine.hyperparameters import HyperParameters

In [18]:
#Run the Random Search:
tuner_search = RandomSearch(build_model, objective = 'val_accuracy', max_trials=5, directory='output', project_name="fashion_MNIST")

In [19]:
#This by default runs for only "2 EPOCHS". we can also change the value if we want.

tuner_search.search(train_images, train_labels, epochs=3, validation_split=0.1)

Trial 5 Complete [00h 15m 05s]
val_accuracy: 0.1054999977350235

Best val_accuracy So Far: 0.9116666913032532
Total elapsed time: 00h 34m 30s


In [21]:
model = tuner_search.get_best_models(num_models=1)[0]

In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 64)        640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 48)        27696     
_________________________________________________________________
flatten (Flatten)            (None, 27648)             0         
_________________________________________________________________
dense (Dense)                (None, 48)                1327152   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                490       
Total params: 1,355,978
Trainable params: 1,355,978
Non-trainable params: 0
_________________________________________________________________


In [23]:
#RETRAIN ALL THE IMAGES CONSIDERING THE ABOVE BEST MODEL:

model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Epoch 4/10
1688/1688 [==============================] - 112s 66ms/step - loss: 0.1564 - accuracy: 0.9425 - val_loss: 0.2528 - val_accuracy: 0.9120
Epoch 5/10
1688/1688 [==============================] - 110s 65ms/step - loss: 0.1184 - accuracy: 0.9564 - val_loss: 0.2881 - val_accuracy: 0.9122
Epoch 6/10
1688/1688 [==============================] - 110s 65ms/step - loss: 0.0871 - accuracy: 0.9690 - val_loss: 0.3214 - val_accuracy: 0.9077
Epoch 7/10
1688/1688 [==============================] - 113s 67ms/step - loss: 0.0623 - accuracy: 0.9773 - val_loss: 0.3545 - val_accuracy: 0.9133
Epoch 8/10
1688/1688 [==============================] - 110s 65ms/step - loss: 0.0464 - accuracy: 0.9830 - val_loss: 0.3628 - val_accuracy: 0.9122
Epoch 9/10
1688/1688 [==============================] - 110s 65ms/step - loss: 0.0384 - accuracy: 0.9862 - val_loss: 0.4048 - val_accuracy: 0.9162
Epoch 10/10
1688/1688 [==============================] - 110s 65ms/step - loss: 0.0274 - accuracy: 0.9905 - val_loss: 

# THIS IS HOW CNN MODEL IS CREATED AND OPTIMIZED USING KERAS

## IF YOU GUYS FINDS THIS INSIGHTFUL, PLEASE GIVE AN UPVOTE.